In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import awkward as ak
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import seaborn as sns

import time

from hist import Hist

import babar_analysis_tools as bat
from analysis_variables import *
import myPIDselector

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, accuracy_score 
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

import joblib
import os

In [ ]:
print(np.__version__)

print(plt.__version__)

In [ ]:
BNC_tag = ""
BNC_bool = False
UNBLINDED_bool = True
if UNBLINDED_bool is True:
    BNC_tag = '_UNBLINDED'

# BNC
#BNC_tag = "_BNC"
#BNC_bool = True

#####################################################################
# Where are we running this?
#####################################################################
## Bellis computer
topdir= "/home/bellis/babar_data_local/bnv_plambda"

if BNC_bool:
    topdir= "/home/bellis/babar_data/bnv_plambda_bnc"

## My laptop
#topdir= "/Users/josieswann/BaBar_analyses/BNV_pLambda/"
#####################################################################


#####################################################################
# Get the BNV data
#####################################################################
#data, data_collision = bat.load_datasets(topdir=topdir, subset='Run1')
data, data_collision = bat.load_datasets(topdir=topdir, subset='all', UNBLINDED=UNBLINDED_bool)

In [ ]:
dataset_information= pd.read_csv("dataset_statistics.csv")
cs_data= pd.read_csv("SP_cross_sections_and_labels.csv")

no_notes= cs_data.drop(["Uncertainty","Note: cross sections found at https://babar-wiki.heprc.uvic.ca/bbr_wiki/index.php/Physics/Cross_sections,_luminosities,_and_other_vital_stats"], axis= 1)
no_notes

sp= data["spmode"]

splist= np.unique(sp.to_list())
splist

'''
dcuts= bat.get_final_masks(data, region_definitions= region_definitions)

print([dcuts.keys()])
print()

for key in dcuts.keys():
    print(f'{key:3d} {dcuts[key]["name"]}')

dcuts[3]
'''

bat.fill_new_entry_with_tag_side_B(data)
data["BtagSideMes"]
bat.fill_new_entry_with_tag_side_B(data_collision)
data_collision["BtagSideMes"]

all_hists= bat.create_empty_histograms(hist_defs)

bkg_spmodes= ["998","1005","3981","1235","1237"]
sig_spmodes= ["-999"]

spmodes= bkg_spmodes+sig_spmodes

weights= {}
for sp in spmodes: 
    weights[sp]= bat.scaling_value(int(sp),dataset_information=dataset_information, cs_data= cs_data, plot= False, verbose= False)
    #weights[sp]=1

weights["-999"]= 1000 #scales signal higher 
weights["0"]= 1 #idk what this is for;;; ASK

print(weights)

### ASK WHAT THESE MEAN
#tag= "EARLY_CUT"
cuts_tag= "CUTS_cuts_1_2_3"


# Generate cuts

In [ ]:

##################################################
# SP
dcuts= bat.get_final_masks(data, region_definitions= region_definitions)



###################################
# Collision
dcuts_col= bat.get_final_masks(data_collision, region_definitions= region_definitions)



In [ ]:
subset = ['spmode', 'BpostFitMes', 'BpostFitDeltaE', 'Lambda0_unc_Mass', \
      'BtagSideMes', 'BSphr', 'BThrust', 'BCosThetaS', \
      'R2', 'R2All', \
      'thrustMag', 'thrustMagAll', 'thrustCosTh', 'thrustCosThAll', 'sphericityAll', \
      'BCosSphr', 'BCosThetaT', 'BCosThrust', 'BLegendreP2', 'BR2ROE', 'BSphrROE', \
      'BThrustROE', 'BcosthCM']


In [ ]:
###################################
# SP
mask_event= dcuts[1]["event"]# & dcuts[2]["event"] & dcuts[3]["event"] 
mask= mask_event

df_sp = bat.dump_awkward_to_dataframe(data[mask], fields_to_dump=subset)#, dropna=True)

# Put the cuts into the dataframe 
cut1 = dcuts[1]['event'] # This is the main cut that gets rid of duplicates
cuts_to_add = [2, 3, 4, 6, -1]
for cut in cuts_to_add:
    bools = dcuts[cut]['event']
    colname = f'cut_{cut}'
    print(colname, len(bools[cut1]), bools[cut1])

    df_sp[colname] = bools[cut1]

# Drop entries where there are nans
print(len(df_sp))
df_sp.dropna(inplace=True)
print(len(df_sp))


In [ ]:
###################################################
# Collision
mask_event= dcuts_col[1]["event"]# & dcuts[2]["event"] & dcuts[3]["event"] 
mask= mask_event

df_col = bat.dump_awkward_to_dataframe(data_collision[mask], fields_to_dump=subset)

# Put the cuts into the dataframe 
cut1 = dcuts_col[1]['event'] # This is the main cut that gets rid of duplicates
cuts_to_add = [2, 3, 4, 6, -1]
for cut in cuts_to_add:
    bools = dcuts_col[cut]['event']
    colname = f'cut_{cut}'
    print(colname, len(bools[cut1]), bools[cut1])

    df_col[colname] = bools[cut1]

# Drop entries where there are nans
print(len(df_col))
df_col.dropna(inplace=True)
print(len(df_col))

In [ ]:
ntrain_sig = 30000
ntrain_bkg = 30000
trial = 15
features = 4

ntrain_tag = f'features_{features}_nsig_{ntrain_sig}_nbkg_{ntrain_bkg}_trial{trial:0d}'

print(ntrain_tag)

# NO BNC tag in the name
model_filename = f"MODEL_MLPClassifier_CUTS_1_2_3_{ntrain_tag}_.pkl"

print(model_filename)
print()

#
if os.path.exists(model_filename):
    print(f"The file '{model_filename}' exists.")
else:
    print(f"The file '{model_filename}' does not exist.")

In [ ]:
# Load in the model
workspace = joblib.load(model_filename)

bat.add_probas_to_dfs(workspace, df_col, df_sp)#.drop(columns=columns_to_drop))


In [ ]:
print(f'{ntrain_tag}{BNC_tag}')

In [ ]:
outfilename = f"DATAFRAME_SP_MODEL_MLPClassifier_CUTS_1_2_3_{ntrain_tag}{BNC_tag}.pkl"
df_sp.to_parquet(outfilename)

outfilename = f"DATAFRAME_COL_MODEL_MLPClassifier_CUTS_1_2_3_{ntrain_tag}{BNC_tag}.pkl"
df_col.to_parquet(outfilename)

# Read in the files and plot